### Модуль 1. О вкусной и здоровой пище

Автор: Руслан Когочкин (https://www.kaggle.com/ruslankogochkin)

Результат, MAE: 0.17850

In [3]:
import pandas as pd
import re
import datetime
from datetime import datetime, timedelta
import math
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

def predict_rating_tripadvisor():
    """Формирует csv-файл с прогнозом рейтинга ресторанов TripAdvisor"""
    df_main = prepare_DF(pd.read_csv('main_task.csv'))
    df_kaggle = prepare_DF(pd.read_csv('kaggle_task.csv'))
    
    X_train = df_main.drop(['Restaurant_id', 'Rating', 'City', 'Cuisine Style', 'Price Range',
                            'Reviews', 'URL_TA', 'ID_TA'], axis = 1)
    y_train = df_main['Rating']
    X_test = df_kaggle.drop(['Restaurant_id', 'Name', 'City', 'Cuisine Style', 'Price Range',
                             'Reviews', 'URL_TA', 'ID_TA'], axis = 1)

    regr = RandomForestRegressor(n_estimators=100)
    regr.fit(X_train, y_train)
    y_pred = round_rating(regr.predict(X_test))
    
    df = pd.DataFrame({'Restaurant_id': df_kaggle['Restaurant_id'], 'Rating': y_pred})
    df.to_csv('rating_tripadvisor.csv', index = False)
    return 'Файл "rating_tripadvisor.csv" создан. Работа завершена.'

def prepare_DF(df):
    """Очищает DataFrame и создает новые признаки"""
    cities = get_cities()
    bad_words = get_bad_words()
    pattern_date = re.compile('\d\d/\d\d/\d\d\d\d')
    df['Price Range'] = df['Price Range'].apply(update_price)
    df = pd.concat([df, pd.get_dummies(df['Price Range'])], axis=1)
    df['Capital'] = df['City'].apply(lambda city: cities[city][0])
    df['Population'] = df['City'].apply(lambda city: cities[city][1])
    df['Square'] = df['City'].apply(lambda city: cities[city][2])
    df = pd.concat([df, pd.get_dummies(df['City'])], axis=1)
    df['Cuisine Style'] = df['Cuisine Style'].apply(update_cuisine_style_to_list)
    df['Cuisine'] = df['Cuisine Style'].apply(count_cuisine_amount)
    df['Top Cuisine'] = df['Cuisine Style'].apply(find_cuisine_popular)
    df['Number of Reviews'] = df['Number of Reviews'].fillna(0)
    df['Reviews'] = df['Reviews'].fillna('[[], []]')
    df['Review_site'] = df['Reviews'].apply(check_review_site, pattern = pattern_date)
    df['Between_reviews'] = df['Reviews'].apply(count_days_between_reviews, pattern = pattern_date)
    df['Freq_reviews'] = df['Between_reviews'].apply(lambda days: 1 if days < 45 else 0) # частота отзывов
    df['Until_today_reviews'] = df['Reviews'].apply(count_days_between_last_review_and_today, pattern = pattern_date)
    df['Reviews'] = df['Reviews'].apply(update_words_in_reviews_to_list)
    df['Bad'] = df['Reviews'].apply(find_words_in_review, words = bad_words)
    return df

def get_cities():
    """Город: столица (0-нет, 1-да), население (млн. человек), площадь (кв. км)"""
    cities = {'Paris':[1, 2.1, 105], 'Stockholm':[1, 0.9, 188], 'London':[1, 8.9, 1572], 'Berlin':[1, 3.7, 891],
    'Munich':[0, 1.4, 310], 'Oporto':[0, 0.2, 41], 'Milan':[0, 1.3, 181], 'Bratislava':[1, 0.4, 368],
    'Vienna':[1, 1.8, 414], 'Rome':[1, 2.8, 1287], 'Barcelona':[0, 5.5, 100], 'Madrid':[1, 6.5, 607],
    'Dublin':[1, 1.3, 318], 'Brussels':[1, 0.2, 33], 'Zurich':[0, 0.4, 92], 'Warsaw':[1, 1.7, 517],
    'Budapest':[1, 1.7, 525], 'Copenhagen':[1, 0.6, 86], 'Amsterdam':[1, 0.8, 219], 'Lyon':[0, 0.5, 48],
    'Hamburg':[0, 1.8, 755], 'Lisbon':[1, 0.5, 100], 'Prague':[1, 1.3, 500], 'Oslo':[1, 0.6, 454],
    'Helsinki':[1, 0.6, 715], 'Edinburgh':[1, 0.5, 118], 'Geneva':[0, 0.5, 16], 'Ljubljana':[1, 0.3, 164],
    'Athens':[1, 0.6, 412], 'Luxembourg':[1, 0.6, 904], 'Krakow':[0, 0.7, 327]}
    return cities

def get_bad_words():
    """Возвращает список плохих слов"""
    bad_words = ['bad', 'poor', 'worst', 'terrible', 'disappointing', 'overpriced',
                 'avoid', 'awful', 'horrible', 'disappointed']
    return bad_words

def update_price(price):
    """Заполняет пропуски в Price Range и меняет старые значения"""
    if pd.isnull(price):
        return 'na' #Т.к. цены не указаны на сайте
    else:
        if price == '$':
            return 'low'
        elif price == '$$ - $$$':
            return 'med'
        else:
            return 'high'

def update_cuisine_style_to_list(cuisine):
    """Преобразует строку cuisine в list"""
    if pd.isnull(cuisine):
        return cuisine
    else:
        cuisine_style = cuisine[1:-1].replace(" '", "").replace("'", "").split(',')
        return cuisine_style

def count_cuisine_amount(cuisine):
    """Считает количество видов кухонь в списке cuisine"""
    if type(cuisine) == list:
        return len(cuisine)
    else:
        return 0 #Т.к. кухни не указаны на сайте

def find_cuisine_popular(cuisine):
    """Ищет самую популярную кухню в списке cuisine"""
    answer = 0
    if type(cuisine) == list:
        for item in cuisine:   
            if item == 'Vegetarian Friendly':
                answer = 1
                break
            else:
                continue
    return answer

def check_review_site(review, pattern):
    """Проверяет есть ли отзыв на сайте.
    Если находит дату в review, значит есть отзыв"""
    if len(pattern.findall(review)) > 0:
        return 1
    else:
        return 0

def count_days_between_reviews(review, pattern):
    """Считает количество дней между двумя отзывами"""
    dates = pattern.findall(review)
    if len(dates) == 2:
        dt0 = datetime.strptime(dates[0], '%m/%d/%Y')
        dt1 = datetime.strptime(dates[1], '%m/%d/%Y')
        if dt0 > dt1:
            delta = dt0 - dt1
        else:
            delta = dt1 - dt0
        return delta.days
    else:
        return 141 # среднее между отзывами 141

def count_days_between_last_review_and_today(review, pattern):
    """Считает дни от последнего отзыва до сегодня"""
    dates = pattern.findall(review)
    today = datetime.today()
    if len(dates) == 2:
        dt0 = datetime.strptime(dates[0], '%m/%d/%Y')
        dt1 = datetime.strptime(dates[1], '%m/%d/%Y')
        if dt0 > dt1:
            delta = today - dt0
        else:
            delta = today - dt1
        return delta.days
    if len(dates) == 1:
        dt0 = datetime.strptime(dates[0], '%m/%d/%Y')
        delta = today - dt0
        return delta.days
    else:
        return 1100 # среднее между отзывами 1100

def update_words_in_reviews_to_list(review):
    """Преобразует строку review в list"""
    words = review[2:].replace("'", "").split('], [')[0]
    words = words.replace(",", "").replace("!", "").lower().split(' ')
    return words  
    
def find_words_in_review(review, words):
    """Проверяет есть ли слово из review в списке words"""
    if type(review) == list:
        for item in review:
            if item in words:
                return 1
            else:
                continue
        return 0
    else:
        return 0    
    
def round_rating(y_pred):
    """Округляет предсказанные значения рейтинга.
    Если дробная часть ближе к 0, то округляет к 0, 1 - к 1,
    Если дробная часть лежит в окрестностях 0.5, то округляет к 0.5"""
    rating = []
    for y in y_pred:
        y = math.modf(y)
        if y[0] < 0.25:
            y = y[1]
        elif y[0] > 0.75:
            y = y[1] + 1
        else:
            y = y[1] + 0.5
        rating.append(y)
    return np.array(rating)

In [6]:
predict_rating_tripadvisor()